In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import scipy.special

In [2]:
def candidate(old):
    splitting_point = None
    adding_point = None
    removing_point = None
    
    new = old.copy()
    if np.random.uniform(0,1,1)>0.5: #split
        choices = [i for i in range(1,len(old) - 1) if old[i-1] == 1 and old[i] == 1 and old[i + 1] == 1]
        if len(choices)>0:
            choice = np.random.choice(choices)
            splitting_point = choice
            new[splitting_point]=0
    else:
        if np.random.uniform(0,1,1)>0.5: #add
            choices = [i for i in range(len(old) - 1) if old[i] == 0 and old[i + 1] == 1]
            if len(choices)>0:
                adding_point = np.random.choice(choices)
                new[adding_point]=1
        else: #remove
            choices = [i for i in range(len(old) - 1) if old[i] == 1 and old[i + 1] == 0]
            if len(choices)>0:
                removing_point = np.random.choice([i for i in range(len(old) - 1) if old[i] == 1 and old[i + 1] == 0])
                new[removing_point]=0
    if splitting_point != None:
               print(f'Splitting in {splitting_point}')
    if adding_point != None:
               print(f'Adding in {adding_point}')
    if removing_point != None:
               print(f'Removing in {removing_point}')
    
    return new

In [5]:
# example
old = [0,0,0,1,1,0,0,1,0,0,0,1,1,1]
print(str(old)+'\n')
for i in range(10):
    new = candidate(old)
    print(str(new)+'\n')
    old = new

[0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1]

Splitting in 12
[0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1]

[0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1]

Adding in 12
[0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1]

Adding in 6
[0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1]

Removing in 4
[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1]

Splitting in 12
[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1]

Removing in 7
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1]

Adding in 10
[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1]

[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1]

[0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1]



In [14]:
# runs counting check
old = [0,0,0,1,1,0,0,1,0,0,0,1,1,1]

def count_runs(vector): #i.e. r(T0)
    interval_count = 0
    in_interval = False
    for value in vector:
        if value == 1:
            if not in_interval:
                interval_count += 1
                in_interval = True
        else:
            in_interval = False
    in_interval = False
    for value in vector:
        if value == 0:
            if not in_interval:
                interval_count += 1
                in_interval = True
        else:
            in_interval = False
    return interval_count

count_runs(old)

6

In [15]:
def PT0(vector): 
    return np.exp(-count_runs(vector))
    
def get_K0(K,T,T0_01):
    idxs = np.where(np.array(T0_01)==1)[0]
    K_tau = K[idxs,:]
    K_tau_tau = K_tau[:,idxs]
    return K - np.dot(np.dot(K_tau.T,np.linalg.inv(K_tau_tau)),K_tau)

def likelihood(vector): 
    K0 = get_K0(K,T,vector)
    def R0(i, j):
        Ri = np.fromfunction(np.vectorize(lambda t: integrate(T,K0[:,t] * x[i,:])), (g,), dtype=int)
        Rij = integrate(T, Ri * x[j,:])
        return  Rij
    
    def M():
        return np.fromfunction(np.vectorize(lambda i, j: R0(i, j)), (n, n), dtype=int)+np.identity(n)
    
    m_matrix = M()
    first = np.linalg.det(m_matrix)**(-0.5)
    
    inv = np.linalg.inv(m_matrix)
    s11 = np.dot(np.dot(np.transpose(np.ones(n)),inv),np.ones(n))
    
    syy = np.dot(np.dot(np.transpose(y),inv),y)
    s1y = np.dot(np.dot(np.transpose(np.ones(n)),inv),y)
    b = 0.5*(syy-s1y**2/s11)
    
    return first/np.sqrt(s11)*(b**(-0.5*(n-1)))

def acceptance_rate(current,new):
    prior = PT0(current)
    prior_new = PT0(new)
    likl = likelihood(current)
    likl_new = likelihood(new)
    
    return (likl_new*prior_new)/(likl*prior)

def new_point(current):
    cand = candidate(current).copy()
    rate = acceptance_rate(current, cand)
    
    if np.random.uniform(0, 1, 1) < rate:
        print("Accepted\n")
        return cand
    else:
        print("Rejected\n")
        return current
    
def chain_builder(initial_points,n_sim,burn):
    chain = [initial_points]
    for i in range(n_sim):
        new = new_point(chain[-1]).copy()
        chain.append(new)
    return chain[burn:]

In [16]:
# is our definition of prior correct?
T =   np.array([4,5,6,7,8,9,10,11])
ex1 = [0,0,1,1,1,1,0,0]
ex2 = [0,1,1,0,0,1,1,0]
PT0(ex1), PT0(ex2)

(0.049787068367863944, 0.006737946999085467)

### Example

In [25]:
g = 128
T = np.linspace(-3,3,g)
step = T[1]-T[0]
n = 30

def integrate(T, f): 
    tot = 0;
    for i in range(len(T)-1):
        tot = tot + (1/2 * (f[i]+f[i+1]) * (T[1]-T[0]))
    return tot

def prior_kernel(s,t):
    return np.exp(-2*((t-s)**2))

K = np.fromfunction(np.vectorize(lambda s, t: prior_kernel(T[s], T[t])), (g, g), dtype=int)

x = np.random.multivariate_normal(np.zeros(g), K, n)

beta = np.sin(np.pi*T/4)

y = np.fromfunction(np.vectorize(lambda i: integrate(T,  x[i,:]*beta)+np.random.normal(0,1,1)), (n,), dtype=int)

In [21]:
# is our definition of likelihood correct?
T01 = list([int(np.random.choice([0,1])) for i in range(len(T))])
T02 = list([int(np.random.choice([0,1])) for i in range(len(T))])

T03 = T02.copy()
T03[0:5]=[0,0,0,0,0] # less correct

likelihood(T01),likelihood(T02),likelihood(T03)

(1.3206740214713857e-22, 8.2404851591593735e-22, 3.5898715197553325e-22)

In [22]:
T0_init = np.ones(g)

In [23]:
chain = chain_builder(T0_init,1000,0)

Accepted

Accepted

Accepted

Splitting in 8
Rejected

Accepted

Splitting in 8
Rejected

Splitting in 12
Rejected

Splitting in 15
Rejected

Accepted

Splitting in 15
Rejected

Splitting in 6
Rejected

Splitting in 14
Rejected

Splitting in 12
Accepted

Adding in 12
Accepted

Accepted

Accepted

Accepted

Accepted

Splitting in 9
Rejected

Accepted

Accepted

Splitting in 12
Rejected

Accepted

Splitting in 5
Accepted

Splitting in 8
Rejected

Removing in 4
Accepted

Splitting in 15
Accepted

Splitting in 2
Rejected

Adding in 15
Accepted

Splitting in 1
Rejected

Splitting in 15
Rejected

Adding in 5
Accepted

Adding in 4
Accepted

Splitting in 13
Rejected

Accepted

Splitting in 16
Accepted

Removing in 15
Accepted

Splitting in 10
Accepted

Removing in 9
Accepted

Adding in 10
Accepted

Splitting in 11
Accepted

Adding in 9
Accepted

Adding in 11
Accepted

Splitting in 5
Rejected

Adding in 16
Rejected

Splitting in 7
Rejected

Adding in 16
Accepted

Splitting in 1
Rejected

Removi

Rejected

Adding in 17
Rejected

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Adding in 17
Accepted

Splitting in 18
Rejected

Adding in 16
Rejected

Splitting in 18
Accepted

Adding in 16
Rejected

Removing in 17
Accepted

Accepted

Accepted

Adding in 18
Accepted

Adding in 17
Rejected

Adding in 17
Rejected

Accepted

Accepted

Accepted

Adding in 17
Rejected

Accepted

Accepted

Accepted

Adding in 17
Rejected

Adding in 17
Rejected

Adding in 17
Rejected

Accepted

Adding in 17
Rejected

Adding in 17
Rejected

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Adding in 17
Rejected

Accepted

Adding in 17
Rejected

Accepted

Accepted

Accepted

Adding in 17
Rejected

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Accepted

Adding in 17
Rejected

Accepted

Adding in 17
Rejected

Adding in

In [24]:
chain[-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 1.])